## Logging and plotting data

In the previous practical sessions we saw how to define, attach, start, stop and detach routines to the agents. Here we are going to see how to use such routines to record data perceived or produced by the agents. This will allow the plotting of figures in the notebook. This can also be useful to better visualize what is happening in your simulation and help you debugging it. 

Let's start with a simple example where we record the values returned by both proximeters through time.

In [ ]:
from vivarium.controllers.notebook_controller import NotebookController
from vivarium.utils.handle_server_interface import start_server_and_interface, stop_server_and_interface

First, start the simulator and create a controller object.

In [ ]:
start_server_and_interface(scene_name="session_4")

In [ ]:
controller = NotebookController()

Then, we will assign variables to each robots present in the environment.

In [7]:
agent_0, agent_1, agent_2 = controller.agents

In [ ]:
controller.run()

Now, let's add some logs to the first agent. Recording data is realized by the `add_log` function of the agent, which require two arguments: the arbitrary label of the recorded data, called the topic, and the data to be recorded. For example:

In [9]:
agent_0.add_log("test", 1)

This stores the data `1` in a topic that we arbitrarily call `"test"`. We can retrieve this data by using the `get_log` function , which requires as argument the name of the topic (`"test"` in this example):

In [ ]:
print(agent_0.get_log("test"))

Calling `agent_0.get_log("test")` returns the list of the data recorded in the topic `"test"` by `agent_0`. Here it prints `[1]`, a list containing the only data we have stored before.

Let's add another data to the same topic:

In [11]:
agent_0.add_log("test", 42)

And retrieve the data recorded on this topic:

In [ ]:
print(agent_0.get_log("test"))

The second value we have added, `42`, has been appended to the list, which now contains the two recorded data.

We can add another value to another topic:

In [13]:
agent_0.add_log("another_topic", 18)

and retrieve it using `get_log`, this time with the name of this new topic:

In [ ]:
print(agent_0.get_log("another_topic"))

Of course the data recorded before in the topic `"test"` is still accessible:

In [ ]:
print(agent_0.get_log("test"))

The names chosen for the topics are completely arbitrary. They are just tags that you choose for organizing the data according to their source.

This two functions allow to record various data from the simulation, organizing them by topics differentiated by their names. Coupled with an appropriate routine running on the agent that continuously calls the `add_log` function, for example to record the values of the proximeters or the motors through time, this can then be used for generating figures plotting what is happening in the simulation.

Let's define a routine that record the values sensed by the left and right proximeters:

In [17]:
# Define a routine using the method we have seen in the last session. Here we call it agent_log 
def agent_log(agent):
    # Retrieve the values of the left and right proximeters:
    left, right = agent.sensors()
    
    # record the left activation in a topic called "left_prox"
    agent.add_log("left_prox", left)
    # record the right activation in a topic called "right_prox"
    agent.add_log("right_prox", right)

Then attach and start it on `agent_0` as usual with the `attach_routine` function. Also set both of its motors to 1 to make it move in the simulation.

In [26]:
agent_0.left_motor = agent_0.right_motor = 1.
# Add an interval argument to only log every 50 timesteps
agent_0.attach_routine(agent_log, interval=50)

We set an interval of 20 for the `agent_log` routine, meaning that the left and right proximeter values will be recorded every 20 iterations in the controller loop. We could put a smaller interval for having more precision, but keep in mind that this could cause memory issues, since the data are recorded continuously at the specified interval. For example, with an interval of 10 and a fps of 20, it will record $(20/10)*60=120$ values each minute of the simulation. So it can take a lot of space in memory if you let the simulation running for a long time.

You can access to the values recorded from the left proximeter with:

In [ ]:
print(agent_0.get_log("left_prox"))

We can get the length of this list with the following command:

In [ ]:
print(len(agent_0.get_log("left_prox")))

This prints all the recorded values for the left proximeter. In case there is too much values to be printed, you can clear the log by executing:

In [30]:
agent_0.clear_all_logs()

This will erase all the data that `agent_0` has recorded. Recording will however still continue to occur because the `agent_log` routine is still runnning. You can stop the routine as usual:

In [31]:
agent_0.detach_routine("agent_log")

Now the data are not recorded anymore.

Let's now define a population of agents foraging for spheres according to their energy level as in the last session. We will also define a routine continuously recording various data during the simulation, for example proximeter activations or energy levels.

First we define the obstacle avoidance behavior for all of the scene obstacles:

In [ ]:
controller.print_subtypes_list()

In [34]:
def obstacle_avoidance(agent):
    left, right = agent.sensors(sensed_entities=["s_obstacles", "b_obstacles"])
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return left_wheel, right_wheel

Then the routine that compute the energy level:

In [35]:
max_energy_level = 100.
init_energy_level = 50.

In [36]:
def foraging_drive(agent): 
    if agent.has_eaten():
        agent.energy_level += 10  # if the agent has eaten a sphere, increase its energy level by 0.2
    else:
        agent.energy_level -= 0.01  # otherwise (nothing eaten), decrease the energy level by 0.01
    # The line below bounds the value of the energy level between 0 and 1
    agent.energy_level = min(max_energy_level, max(agent.energy_level, 0.))

Then we define the foraging behavior, which is weighted according to the energy level (due to the last returned value, here `1 - agent.energy_level` because we want the foraging behavior to be more activated when the energy level is lower:

In [37]:
def foraging(agent):
    left, right = agent.sensors(sensed_entities=["resources"])
    left_activation = right
    right_activation = left
    return left_activation, right_activation

And finally we define the routine that will log the data we are interested in (here the left and right activations of the proximeters and the wheels, as well as the energy level of the agent):

In [38]:
# Define a routine using the method we have seen in the last session. Here we call it agent_log 
def agent_log(agent):
    # Retrieve the values of the left and right proximeters:
    left, right = agent.sensors()
    
    # record the left proximeter activation in the topic called "left_prox"
    agent.add_log("left_prox", left)
    # record the right proximeter activation in the topic called "right_prox"
    agent.add_log("right_prox", right)
    # record the energy level in the topic called "energy"
    agent.add_log("energy", agent.energy_level)

Now we can attach and start all the behaviors and routines we have just define on the two agents:

In [47]:
# First start sphere apparition in the environment:
controller.start_resources_apparition(interval=50)
controller.start_eating_mechanism(interval=30)

# For all agents
for e in controller.agents:
    # Detach all existing behaviors and routines:
    e.detach_all_behaviors()
    e.detach_all_routines()
    
    e.diet = ["resources"]
    
    # Attach the two behaviors we have defined
    e.attach_behavior(obstacle_avoidance)
    e.attach_behavior(foraging)
    
    # Attach the routines for recording data and for computing the energy level
    e.attach_routine(agent_log, interval=5)
    e.attach_routine(foraging_drive)
    
    # Set the initial energy level
    e.energy_level = init_energy_level
    
    # start all behaviors and all routines
    e.start_all_behaviors()


This will start the defined behaviors and routines on the two agents. The `agent_log` routine will record the proximeter activations, as well as the energy level of each agent. Using the produced log, we can now plot those data against time. Let's for example plot the activation of the left proximeter through time. This can be done like this:

In [48]:
import matplotlib.pyplot as plt

In [ ]:
# The line below is mandatory to inform the notebook we want to plot directly in it
%matplotlib inline

# Plot the left proximeter value recorded by `agent_0`
plt.plot(agent_0.get_log("left_prox"))
plt.show()

The above figure plot all the left proximeter values recorded from `agent_1`. The x-axis corresponds to the time step of the recording values and the y-axis corresponds to the value of the left proximeter at each time step. The time step depends on the frequency at which we have run the `agent_log` primitive. Above we have set it to 1Hz (one value recorded per second), so in the figure the x-axis represents seconds. If we would have set the frequency of the `agent_log` routine to 2Hz, each unit on the x-axis would then correspond to half a second.

We can indicate the labels of the x and y axes and provide a title for the figure with:

In [ ]:
%matplotlib inline

# Plot the left proximeter value recorded by `agent_0` and set labels to the x and y axes, as well as a title
plt.plot(agent_0.get_log("left_prox"))
plt.xlabel("Time")
plt.ylabel("Left proximeter")
plt.title("Plot of left proximeter activation against time")
plt.show()

This write the labels and the title on the figure.

Let's now plot the energy level of `agent_1`:

In [ ]:
# The line below is mandatory to inform the notebook we want to plot directly in it
%matplotlib inline

# Plot the energy level recorded by `agent_0`
plt.plot(agent_0.get_log("energy"))
plt.xlabel("Time")
plt.ylabel("Energy level")
plt.title("Plot of energy level against time")
plt.show()

We can also plot two time series on the same plot. Let's plot the energy levels of both agents:

In [ ]:
# The line below is mandatory to inform the notebook we want to plot directly in it
%matplotlib inline

# Plot the energy levels recorded by `agent_0` and `agent_2`
plt.plot(agent_0.get_log("energy"))
plt.plot(agent_2.get_log("energy"))
plt.xlabel("Time")
plt.ylabel("Energy level")
plt.title("Plot of energy level against time")
plt.show()

We can add a legend to indicate which line corresponds to which agent:

In [ ]:
# The line below is mandatory to inform the notebook we want to plot directly in it
%matplotlib inline

# Plot the energy levels recorded by `agent_0` and `agent_2`
plt.plot(agent_0.get_log("energy"))
plt.plot(agent_2.get_log("energy"))
plt.legend(["agent_0", "agent_2"])

plt.xlabel("Time")
plt.ylabel("Energy level")
plt.title("Plot of energy level against time")
plt.show()

By right-clicking on the figure and choosing "Save image as", you can store it as a PNG image. This is useful if you want to save it for other purposes.

That's all. Don't forget to properly close the session when you have finished with the notebook:

In [ ]:
stop_server_and_interface()
controller.stop()